# Interactive Nodes

This class of node does not use the typical communication mechanism.
In other nodes, inputs are provided as function arguments to the encapsulated block,
the block is run and its return value is used as output and passed to other nodes.
In an Interactive Node, data flow is managed from inside the block.
This allows you to send and receive data at any point in the code.
This is best demonstrated with an example.

First, import the library:

In [1]:
import deltalanguage as dl

## Defining interactive blocks

The block inside an interactive node is called a *process function*.
This is a function that takes the node itself as argument, and uses the
`send` and `receive` methods on the node to perform input and output:

- `receive` can be called with no arguments, or with an arbitrary number of string arguments.
If called with no arguments, it will retrieve all the inputs and return them to the caller as a dict (from name to value).
If called with string arguments, it will only retrieve the inputs with the specified keys.
If there is just one input, it is returned.
Otherwise, they are packed in a dict as above.

- `send` takes as argument one object that will be sent to the next nodes in the graph.


## The `Interactive` decorator

To turn such a function into a Delta Block, use the `@Interactive` decorator.
It takes three arguments:

- the input types, which is a dict mapping the names of the inputs the process expects to their types.
- the output type
- optionally, the name of the resulting node

Here is a simple Interactive block definition:

In [2]:
@dl.Interactive({'a': str}, int)
def str_to_int(node):
    for i in range(10):
        a = node.receive('a')
        a = int(a)
        node.send(a + i)

    raise dl.DeltaRuntimeExit

Since `a` is a non-optional input, the above block's execution
will pause until an input `a` is available.
Then, the input is parsed as an integer, added a number and sent forward.
This procedure is repeated multiple times and then the node exits.

To see the block in action, we must put it in a DeltaGraph:

In [3]:
s = dl.lib.StateSaver(int, verbose=True)

with dl.DeltaGraph() as graph_1:
    num = str_to_int.call(a='1')
    s.save(num)

print(graph_1)

DeltaGraph[main] {
    str_to_int_0:
        in : (a, DStr8192, 0, False)
        out: (save_2.val, DInt32)
    node_1:
        out: (str_to_int_0.a, DStr8192)
    save_2:
        in : (val, DInt32, 0, False)
}



Let us examine the graph, there are three nodes:

- our interactive process, which has one input `a` and one output going to `printer.n`

- a node that outputs a string to the input `a` of `str_to_int`

- the `printer` node

This _looks_ good, so we can make a runtime for the graph and run it.

In [4]:
s.reset()

rt = dl.DeltaPySimulator(graph_1)
rt.run()

# We cannot say for sure how many messages are received
assert len(s.saved) in range(11)

If you are lucky you will see 10 messages from the printer, but if the previous cell
is executed several time you should see that it's not always true.

This example shows a fundamental property of `DeltaGraph`, namely each node is executed
independently as long as there is no dependency on external data.
In this example `DeltaPySimulator` might reach
`DeltaRuntimeExit` before each printer node outputs the result.
This is not fixed by
`DeltaPySimulator` and it meant to be like this, at least at the moment.

To resolve this inconsistency one needs to make sure that `DeltaRuntimeExit`
is reached only after a succesful execution of each printer.

For instance, let's add a status return from the printer. This creates
a loop, which can be resolved via placeholder covered in another tutorial.

In [5]:
@dl.Interactive({'a': str, 'status': bool}, int)
def str_to_int_with_status(node):
    for i in range(10):
        a = node.receive('a')
        a = int(a, base=10)
        node.send(a + i)
        if node.receive('status'):
            print(f"INTERACTIVE NODE: STATUS OK")

    raise dl.DeltaRuntimeExit

with dl.DeltaGraph() as graph_2:
    ph = dl.placeholder_node_factory()
    num = str_to_int_with_status.call(a='1', status=ph)
    st = s.save_and_ack(num)
    ph.specify_by_node(st)

print(graph_2)

DeltaGraph[main] {
    str_to_int_with_status_3:
        in : (a, DStr8192, 0, False)
        in : (status, DBool, 0, False)
        out: (save_and_ack_5.val, DInt32)
    node_4:
        out: (str_to_int_with_status_3.a, DStr8192)
    save_and_ack_5:
        in : (val, DInt32, 0, False)
        out: (str_to_int_with_status_3.status, DBool)
}



In [6]:
s.reset()

rt = dl.DeltaPySimulator(graph_2)
rt.run()

assert s.saved == list(range(1, 11))

saving 1
INTERACTIVE NODE: STATUS OK
saving 2
INTERACTIVE NODE: STATUS OK
saving 3
INTERACTIVE NODE: STATUS OK
saving 4
INTERACTIVE NODE: STATUS OK
saving 5
INTERACTIVE NODE: STATUS OK
saving 6
INTERACTIVE NODE: STATUS OK
saving 7
INTERACTIVE NODE: STATUS OK
saving 8
INTERACTIVE NODE: STATUS OK
saving 9
INTERACTIVE NODE: STATUS OK
saving 10
INTERACTIVE NODE: STATUS OK


Now we can see a consistent result with 10 outputs.

## A more complicated example

Now let us see what Interactive Nodes can be used for in a more
complex situation.
This next example will also take advantage of the ability to receive
different inputs at different times.

In [7]:
TwoIntsT, TwoInts = dl.make_forked_return({'x': int, 'y': int})


@dl.Interactive({'thr': int, 'num': int}, TwoIntsT)
def add_to_threshold(node):
    """Sends num on the left port until it is greater than the threshold,
    then sends it on the right port.
    """
    threshold = node.receive('thr')
    num = 1

    while num < threshold:
        node.send(TwoInts(num, None))
        num = node.receive('num')
    node.send(TwoInts(None, num))

Let us define another block that adds two numbers together:

In [8]:
@dl.DeltaBlock()
def add(x: int, y: int) -> int:
    return x + y

And now we can wire all of these into a graph.
We will create a cycle, so placeholders are needed.
A placeholder can be satisfied with an interactive process with
the `specify_by_process` method.
It can optionally take additional keyword arguments to wire as
inputs to the process.

In [9]:
with dl.DeltaGraph() as graph_3:
    ph = dl.placeholder_node_factory()

    # connect the "right" port of the interactive node to the printer
    s.save_and_exit(ph.y)

    # connect the right one to addition
    added = add(x=ph.x, y=2)

    # complete the cycle by specifying the placeholder
    ph.specify_by_process(add_to_threshold, num=added, thr=10)

# Let us see the graph
print(graph_3)

DeltaGraph[main] {
    save_and_exit_6:
        in : (val, DInt32, 0, False)
    node_7:
        out: (add_8.y, DInt32)
    add_8:
        in : (x, DInt32, 0, False)
        in : (y, DInt32, 0, False)
        out: (add_to_threshold_9.num, DInt32)
    add_to_threshold_9:
        in : (thr, DInt32, 0, False)
        in : (num, DInt32, 0, False)
        out: (save_and_exit_6.val, DInt32)
        out: (add_8.x, DInt32)
    node_10:
        out: (add_to_threshold_9.thr, DInt32)
}



This graph is more complicated:
- One output of the interactive node (`add_until`) goes into `add`,
and the other goes to the printer.

- `add` takes one more input from a constant node (the 2 argument).

- `add_until` takes the result of `add` as input, and has the
`thr` input from a constant node (the 10).

- This will create a feedback loop that will keep incrementing `num`
in steps of 2, until it is greater than `thr`.

Let us see what happens when we run it.

In [10]:
s.reset()

rt = dl.DeltaPySimulator(graph_3)
rt.run()

assert s.saved == [11]

saving 11


We started with 1, and kept adding 2 until the number 11 is reached.
At that point, the condition `num > threshold` was true and the number
was sent to the printer instead of to `add`.

## Conclusion

Interactive nodes can be useful for algorithms that require multiple input
and output points.
In a quantum computing context, an algorithm that dynamically adjusts
the number of measurements of an operator based on the uncertainty could
be encapsulated into this type of node.